In [1]:
t = sp.Symbol('t')
q1 = sp.Function("q1")(t)
q2 = sp.Function("q2")(t)
q1_dot = sp.diff(q1, t)
q2_dot = sp.diff(q2, t)
q1_ddot = sp.diff(q1_dot, t)
q2_ddot = sp.diff(q2_dot, t)

# Generalized coordinates and velocities
q = [q1, q2]
q_dot = sp.Matrix([q1_dot, q2_dot])
q_ddot = sp.Matrix([q1_ddot, q2_ddot])

# Define the link lengths and masses
m1, m2, g = sp.symbols('m1 m2 g')
l1, l2, lc1, lc2 = sp.symbols('l1 l2 lc1 lc2')
J1, J2 = sp.symbols('J1 J2')

In [2]:
from sympy.printing.str import StrPrinter

class CustomStrPrinter(StrPrinter):
    def _print_Pow(self, expr):
        base, exp = expr.as_base_exp()
        if exp == 1:
            return self._print(base)
        elif exp == -1:
            return f"1/({self._print(base)})"
        else:
            return f"{self._print(base)}^{self._print(exp)}"

# Use the custom printer
def custom_pretty(expr):
    return CustomStrPrinter().doprint(expr)

In [3]:
# Actuator parameters
xa, ya = sp.symbols('xa ya') #x, y coordinate of actuator

# Rope connection length along joint 2
lr2 = sp.symbols('lr2')
kx = l1 * sp.cos(q1) + lr2 * sp.cos(q2)
ky = l1 * sp.sin(q1) + lr2 * sp.sin(q2)

k = sp.Matrix([[kx],
               [ky]])

dkdq1 = sp.diff(k, q1)
dkdq2 = sp.diff(k, q2)

R = sp.Matrix([[xa - kx],
               [ya - ky]])
Rx = R[0]
Ry = R[1]
r = R/R.norm()

l = sp.sqrt((xa - kx)**2 + (ya - ky)**2)
A1 = sp.diff(l, q1).simplify()
A2 = sp.diff(l, q2).simplify()
sp.pprint(A1, wrap_line = False)
sp.pprint(A2, wrap_line = False)

A = sp.Matrix([[A1],
               [A2]])

A = sp.simplify(A)
#print(custom_pretty(A))
#print("\n")
#sp.pprint(A, wrap_line=False)

            l₁⋅(-lr₂⋅sin(q₁(t) - q₂(t)) + xa⋅sin(q₁(t)) - ya⋅cos(q₁(t)))            
────────────────────────────────────────────────────────────────────────────────────
   _________________________________________________________________________________
  ╱                                      2                                        2 
╲╱  (l₁⋅sin(q₁(t)) + lr₂⋅sin(q₂(t)) - ya)  + (l₁⋅cos(q₁(t)) + lr₂⋅cos(q₂(t)) - xa)  
            lr₂⋅(l₁⋅sin(q₁(t) - q₂(t)) + xa⋅sin(q₂(t)) - ya⋅cos(q₂(t)))             
────────────────────────────────────────────────────────────────────────────────────
   _________________________________________________________________________________
  ╱                                      2                                        2 
╲╱  (l₁⋅sin(q₁(t)) + lr₂⋅sin(q₂(t)) - ya)  + (l₁⋅cos(q₁(t)) + lr₂⋅cos(q₂(t)) - xa)  


In [7]:
h1 = sp.atan2(Rx, Ry)
h1 = l
h2 = q1
theta = sp.Matrix([[h1],
                   [h2]])

Jh = theta.jacobian(q)
Jh = sp.simplify(Jh)
sp.pprint(Jh, wrap_line = False)

⎡            l₁⋅(-lr₂⋅sin(q₁(t) - q₂(t)) + xa⋅sin(q₁(t)) - ya⋅cos(q₁(t)))                          lr₂⋅(l₁⋅sin(q₁(t) - q₂(t)) + xa⋅sin(q₂(t)) - ya⋅cos(q₂(t)))             ⎤
⎢────────────────────────────────────────────────────────────────────────────────────  ────────────────────────────────────────────────────────────────────────────────────⎥
⎢   _________________________________________________________________________________     _________________________________________________________________________________⎥
⎢  ╱                                      2                                        2     ╱                                      2                                        2 ⎥
⎢╲╱  (l₁⋅sin(q₁(t)) + lr₂⋅sin(q₂(t)) - ya)  + (l₁⋅cos(q₁(t)) + lr₂⋅cos(q₂(t)) - xa)    ╲╱  (l₁⋅sin(q₁(t)) + lr₂⋅sin(q₂(t)) - ya)  + (l₁⋅cos(q₁(t)) + lr₂⋅cos(q₂(t)) - xa)  ⎥
⎢                                                                                                                                      

In [8]:
Jh_inv = Jh.inv()
Jh_invtrans = Jh_inv.transpose()

In [9]:
A_theta = Jh_invtrans * A
A_theta = sp.simplify(A_theta)
sp.pprint(A_theta, wrap_line = False)

⎡1⎤
⎢ ⎥
⎣0⎦
